In [10]:
import minerl
from collections import defaultdict

import matplotlib.pyplot as plt
from matplotlib.pyplot import figure
from skimage import color, img_as_ubyte
import gym
from xvfbwrapper import Xvfb

from models import WorldModel
from utils import set_config, get_config, minerl_segs_to_batch
from video import export_video

import torch
from torch import optim
import numpy as np

import os.path as osp
from garage import StepType, EpisodeBatch

%matplotlib inline

In [2]:
scaler = torch.cuda.amp.GradScaler(enabled=True)

In [3]:
def train_world_model_once(world_model, world_optimizer,
                           obs, actions, rewards, discounts):

    
    world_optimizer.zero_grad()


    out = world_model(obs, actions)
    loss, loss_info = world_model.loss(out, obs, rewards, discounts)

    loss.backward()
    world_optimizer.step()

    print('.')

    return out

In [40]:
set_config('debug')

In [2]:
MINERL_DATASET = '/data/umihebi0/users/james/MineRL'

In [ ]:
 with torch.no_grad():
        for i, ep in enumerate(eps.split()):
            obs = (torch.tensor(eps.observations).type(torch.float)
                   .unsqueeze(1)).to(global_device()) / 255 - 0.5
            actions = (torch.tensor(env_spec.action_space.flatten_n(eps.actions))
                       .type(torch.float)).to(global_device())
            steps, channels, height, width = obs.shape
            embedded_observations = world_model.image_encoder(obs[:5])

            # Run first five steps with observations
            out = world_model.observe(embedded_observations[:5].unsqueeze(0),
                                      actions[:5].unsqueeze(0))
            
            recon_latent_states = out['latent_states'].reshape(
                5, world_model.latent_state_size)

            initial_stoch = out['posterior_samples'][:1, -1]
            inital_deter = out['deters'][:1, -1]
            _, imagined_latent_states, _, _ = (
                world_model.imagine(initial_stoch, inital_deter,
                                    actions=actions[5:].unsqueeze(1))
            )
            imagined_latent_states = imagined_latent_states.reshape(
                -1, world_model.latent_state_size)
            latent_states = torch.cat(
                [recon_latent_states, imagined_latent_states], dim=0)

            image_recon = world_model.image_decoder(latent_states).reshape(
                steps, channels, height, width).cpu().numpy()

            image_recon = np.transpose(image_recon, (0, 2, 3, 1))
            original_obs = np.transpose(obs.cpu().numpy(), (0, 2, 3, 1))

            if original_obs.shape[-1] == 1:
                image_recon = np.tile(image_recon, (1, 1, 1, 3))
                original_obs = np.tile(original_obs, (1, 1, 1, 3))

            original_obs = (original_obs + 0.5) * 255
            image_recon = np.clip((image_recon + 0.5) * 255, 0, 255)
            original_obs = original_obs.astype(np.uint8)
            image_recon = image_recon.astype(np.uint8)

            side_by_side = np.concatenate([original_obs, image_recon], axis=2)

            fname = osp.join(path, f'imagined_{itr}_{i}.mp4')
            export_video(
                frames=side_by_side[:, ::-1],
                fname=fname,
                fps=10
            )

            wandb.log({
                os.path.basename(fname): wandb.Video(fname),
            }, step=itr)

In [7]:
with torch.no_grad():
    log_reconstructions(data, data._env_spec, world_model, 0, './videos')

NameError: name 'data' is not defined

In [5]:
data = minerl.data.make('MineRLTreechop-v0', data_dir=MINERL_DATASET)
state, player_action, reward_from_action, next_state, is_next_state_terminal = next(data.load_data(data.get_trajectory_names()[0]))

  0%|          | 0/2001 [00:00<?, ?it/s]
Process ForkPoolWorker-6:
Process ForkPoolWorker-7:
Traceback (most recent call last):
Traceback (most recent call last):
Process ForkPoolWorker-8:
Process ForkPoolWorker-5:
  File "/home/mil/james/miniconda3/envs/minerl/lib/python3.8/multiprocessing/process.py", line 313, in _bootstrap
    self.run()
  File "/home/mil/james/miniconda3/envs/minerl/lib/python3.8/multiprocessing/process.py", line 108, in run
    self._target(*self._args, **self._kwargs)
  File "/home/mil/james/miniconda3/envs/minerl/lib/python3.8/multiprocessing/process.py", line 313, in _bootstrap
    self.run()
  File "/home/mil/james/miniconda3/envs/minerl/lib/python3.8/multiprocessing/pool.py", line 114, in worker
    task = get()
  File "/home/mil/james/miniconda3/envs/minerl/lib/python3.8/multiprocessing/process.py", line 108, in run
    self._target(*self._args, **self._kwargs)
Traceback (most recent call last):
  File "/home/mil/james/miniconda3/envs/minerl/lib/python3.8/m

In [ ]:
def log_reconstructions(data, env_spec, world_model, itr, path, n=3):
    obs, actions, rewards, discounts = segs
    
    n, seq, channels, height, width = obs.shape
    embedded = world_model.image_encoder(obs.reshape(n*seq, channels, height, width))
    out = world_model.observe(embedded.reshape(n, seq, -1), actions)
    image_recon = world_model.image_decoder(
        out['latent_states'].reshape(n*seq, -1),
    ).reshape(n, seq, channels, height, width).cpu().numpy()
    
    image_recon = np.clip(
        (np.transpose(image_recon, (0, 1, 3, 4, 2)) + 0.5) * 255,
        0, 255).astype(np.uint8)
    original_obs = ((np.transpose(obs.cpu().numpy(), (0, 1, 3, 4, 2)) + 0.5) * 255).astype(np.uint8)
    side_by_side = np.concatenate([original_obs, image_recon], axis=3)
    
    for i, img in enumerate(side_by_side):
        fname = osp.join(path, f'reconstructed_{itr}_{i}.mp4')
        export_video(
            frames=img[:, ::-1],
            fname=fname,
            fps=10
        )

        wandb.log({
            os.path.basename(fname): wandb.Video(fname),
        }, step=itr)

In [7]:
data._env_spec.action_space.flat_map(player_action)

array([1. , 0. , 1. , 0. , 0.5, 0.5, 0. , 1. , 1. , 0. , 1. , 0. , 1. ,
       0. , 1. , 0. , 1. , 0. ])

In [20]:
len(data.get_trajectory_names())

209

In [44]:
def sample_minerl_segments(data, n=1, segment_length=50):
    episodes = np.random.choice(len(data), size=n)
    observations = []
    actions = []
    rewards = []
    dones = []
    for i in episodes:
        ep = data[i]
        length = len(ep['observations'])
        start = min(np.random.randint(length),
                    length-segment_length)
        end = start + segment_length
        
        observations.append(ep['observations'][start:end])
        actions.append(ep['actions'][start:end])
        rewards.append(ep['rewards'][start:end])
        dones.append(ep['dones'][start:end])
        
    device = 'cpu'

    obs = torch.tensor(
        np.transpose(observations, (0, 1, 4, 2, 3)) / 255 - 0.5,
        device=device,
        dtype=torch.float)

    actions = torch.tensor(
        np.array(actions),
        device=device,
        dtype=torch.float
    )

    rewards = torch.tensor(np.array(rewards), device=device, dtype=torch.float)

    discounts = torch.tensor(1-np.array(dones), device=device, dtype=torch.float)

    return obs, actions, rewards, discounts

In [29]:
obs, actions, rewards, discounts = sample_segments(episodes, n=3)

torch.Size([3, 50, 3, 64, 64])

In [21]:
episodes = []

observations = []
actions = []
rewards = []
dones = []

for i, name in enumerate(data.get_trajectory_names()):
    ep = defaultdict(list)
    for state, action, reward, next_state, is_next_state_terminal in data.load_data(name):
        ep['observations'].append(next_state['pov'])
        ep['actions'].append(data._env_spec.action_space.flat_map(action))
        ep['rewards'].append(reward)
        ep['dones'].append(is_next_state_terminal)
    episodes.append(ep)
    
    if i % 50 == 0:
        print(i)

100%|██████████| 1985/1985 [00:00<00:00, 13188.51it/s]


0


100%|██████████| 1379/1379 [00:00<00:00, 12005.12it/s]


50


100%|██████████| 1672/1672 [00:00<00:00, 12434.00it/s]


...

100%|██████████| 2468/2468 [00:00<00:00, 12001.91it/s]


150


100%|██████████| 2146/2146 [00:00<00:00, 12982.11it/s]


200


100%|██████████| 1669/1669 [00:00<00:00, 13573.25it/s]


In [22]:
import pickle

In [23]:
with open('minerldata.pickle', 'wb') as f:
    pickle.dump(episodes, f)

In [ ]:
episodes

In [15]:
np.asarray(observations).shape

(1668, 64, 64, 3)

In [41]:
world_model = WorldModel(data._env_spec, config=get_config())

world_optimizer = optim.Adam(
    world_model.parameters(),
    lr=get_config().world.lr,
    eps=get_config().world.eps,
    weight_decay=get_config().world.wd)


/home/mil/james/miniconda3/envs/minerl/lib/python3.8/site-packages/gym/logger.py:30: UserWarning: WARN: Box bound precision lowered by casting to float32
  warnings.warn(colorize('%s: %s'%('WARN', msg % args), 'yellow'))


In [56]:
def log_imagined_rollouts(segs,
                          env_spec,
                          world_model,
                          itr,
                          path):
    
    obs, actions, rewards, discounts = segs
    n, seq, channels, height, width = obs.shape
    
    embedded = world_model.image_encoder(
        obs.reshape(n*seq, channels, height, width)
    ).reshape(n, seq, -1)

    out = world_model.observe(embedded[:,:5],
                              actions[:,:5])
    
    initial_stoch = out['posterior_samples'][:, -1]
    inital_deter = out['deters'][:, -1]
    _, imagined_latent_states, _, _ = (
        world_model.imagine(initial_stoch, inital_deter,
                            actions=torch.swapaxes(actions, 0, 1)[5:])
    )
    imagined_latent_states = torch.swapaxes(imagined_latent_states, 0, 1)
    
    latent_states = torch.cat([
        out['latent_states'], imagined_latent_states
    ], dim=1)
    
    n, seq, feat = latent_states.shape
    
    image_recon = world_model.image_decoder(
        latent_states.reshape(n*seq, feat)).reshape(n, seq, channels, height, width).cpu().numpy()

    image_recon = np.transpose(image_recon, (0, 1, 3, 4, 2))
    original_obs = np.transpose(obs.cpu().numpy(), (0, 1, 3, 4, 2))

    original_obs = (original_obs + 0.5) * 255
    image_recon = np.clip((image_recon + 0.5) * 255, 0, 255)
    original_obs = original_obs.astype(np.uint8)
    image_recon = image_recon.astype(np.uint8)

    side_by_side = np.concatenate([original_obs, image_recon], axis=3)

    for i, img in enumerate(side_by_side):
        fname = osp.join(path, f'imagined_{itr}_{i}.mp4')
        export_video(
            frames=img[:, ::-1],
            fname=fname,
            fps=10
        )

In [45]:
segs = sample_minerl_segments(episodes, n=3, segment_length=50)

In [57]:
with torch.no_grad():
    log_imagined_rollouts(segs, data._env_spec, world_model, 0, './videos')

In [35]:
for _ in range(30):
    obs, actions, rewards, discounts = sample_minerl_segments(episodes, n=50, segment_length=50)
    print(_)


#     logger.log(tabular)
#     logger.dump_all(trainer.step_itr)
#     tabular.clear()
#     trainer.step_itr += 1


0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
